In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

# 加载数据集
df = pd.read_csv("data/train.csv")
plt.imshow(df.iloc[10, 1:].values.reshape(28, 28), cmap="gray")

In [15]:
# 划分特征和标签
X = df.drop("label", axis=1)  # 特征
y = df["label"]  # 标签
# 将数据集按7:3划分为训练数据与测试数据
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)
knn = KNeighborsClassifier(n_neighbors=5)
# 模型训练
knn.fit(x_train, y_train)
# 模型评估，计算准确率
knn.score(x_test, y_test)

In [ ]:
import joblib

joblib.dump(knn, "knn_digit_recognizer")

In [ ]:
knn_loaded = joblib.load("knn_digit_recognizer")
plt.imshow(x_test.iloc[10].values.reshape(28, 28), cmap="gray")
knn.predict(x_test.iloc[10].values.reshape(1, -1))